## Ejemplo 1: Barra sin disipación

In [ ]:
import numpy as np
import plotly.express as px

# ----------- 1. Datos de la barra -----------
L = 1.0
W = 0.1
H = 0.1

# Propiedades del material
k = 1  # Conductividad en W/m·K

# Sin disipación
q = 0 / (L * W * H)  # W/m^3 
# -----------------------------------------

# ------ 2. Condiciones de contorno ------
T1 = 0
T2 = 100
# -------------------------------------


# ------ 3. Cálculo temperatura analítica --------
def T_disipacion(x):
    """Calcula la temperatura en un punto x a lo largo de la barra cuando hay disipación."""
    return T1 + (T2 - T1) * (x / L) + (q / (2 * k)) * (L * x - x**2)

# Puntos a lo largo de la barra
x = np.linspace(0, L, 100)
T_analitica = T_disipacion(x)
# ---------------------------------------------



In [ ]:
# ------- 6. Método de diferencias finitas ------

Nx = 5

# Empezamos con una matriz de ceros
K = np.zeros((Nx, Nx))

# Rellenamos la diagonal principal con -2
np.fill_diagonal(K, -2)

# Rellenamos la diagonal superior con 1
np.fill_diagonal(K[:-1, 1:], 1)

# Rellenamos la diagonal inferior con 1
np.fill_diagonal(K[1:, :-1], 1)


# Condiciones de contorno
K[0,:] = 0
K[0,0] = 1

K[-1,:] = 0
K[-1,-1] = 1


# Vector de términos independientes
Q = np.zeros(Nx)
Q[0] = T1
Q[-1] = T2

# ------------------------------------

# Mostramos la matriz de conductancias y el vector Q
print("Matriz de conductancias K:")
print(K)
print("Vector Q:")
print(Q)




In [ ]:

# ------- 7. Resolvemos el sistema de ecuaciones ------

T_diff_finitas = np.linalg.solve(K, Q)


# ------- 8. Pintamos los resultados ------

# Solucion analítica
fig = px.line(
    x=x,
    y=T_analitica,
    labels={"x": "Posición a lo largo de la barra (m)", "y": "Temperatura (°C)"},
    title="Distribución de temperatura a lo largo de la barra sin disipación",
)

# Datos de diferencias finitas
fig.add_scatter(
    x=np.linspace(0, L, Nx),
    y=T_diff_finitas,
    mode="markers",
    name="Diferencias Finitas"
)

fig.show()
# ---------------------------------------

## Ejemplo 2: Barra con disipación

In [ ]:

# ------ Solución analítica con disipación --------
q = 10 / (L * W * H)  # W/m^3
T_analitica = T_disipacion(x)
# ---------------------------------------------


# ------- Resolvemos el sistema de ecuaciones ------
dx = L / (Nx - 1)
Q = np.ones(Nx)*(-q/k)*dx**2 # Añadimos la disipación
Q[0] = T1
Q[-1] = T2
T_diff_finitas = np.linalg.solve(K, Q)

# ------------------------------------



# ------- Pintamos los resultados ------

# Solucion analítica
fig = px.line(
    x=x,
    y=T_analitica,
    labels={"x": "Posición a lo largo de la barra (m)", "y": "Temperatura (°C)"},
    title="Distribución de temperatura a lo largo de la barra con disipación",
)

# Datos de diferencias finitas
fig.add_scatter(
    x=np.linspace(0, L, Nx),
    y=T_diff_finitas,
    mode="markers",
    name="Diferencias Finitas"
)

fig.show()
# ---------------------------------------

## Ejemplo 3: Barra Aislada por un extremo con disipación

# 

In [ ]:
# ------ Solución analítica ------
q = 1 / (L * W * H)  # W/m^3

def T_disipacion_un_extremo_aislado(x):
    """Calcula la temperatura en un punto x a lo largo de la barra cuando un extremo está aislado y hay disipación."""
    return (q/k)*(L*x - (x**2)/2) + T1

T_analitica = T_disipacion_un_extremo_aislado(x)

# ---------------------------------------------





# ---- Diferencias finitas, construcción de la matriz K -----
Nx = 5
dx = L / (Nx - 1)
K = np.zeros((Nx, Nx))
np.fill_diagonal(K, -2)

np.fill_diagonal(K[:-1, 1:], 1)
np.fill_diagonal(K[1:, :-1], 1)


# Condiciones de contorno izquierda
K[0,:] = 0
K[0,0] = 1
# ------------------------------------------------------------


# ------- Diferencias finitas atrasadas ------
# Tn = Tn-1
K[-1,:] = 0
K[-1,-1] = 1
K[-1,-2] = -1

Q = np.ones(Nx)*(-q/k)*dx**2 # Añadimos la disipación
Q[0] = T1
Q[-1] = 0

T_diff_finitas_atrasadas = np.linalg.solve(K, Q)
# ------------------------------------

# ------- Diferencias finitas centradas ------
# Tn = Tn-1 + q*dx^2/2k
K[-1,:] = 0
K[-1,-1] = -1
K[-1,-2] = 1
Q[-1] = -((q/k)*dx**2)/2

T_diff_finitas_centradas = np.linalg.solve(K, Q)
# ------------------------------------

# ------- Diferencias finitas adelantadas ------
# Tn-1 -Tn = -q*dx^2/2k
K[-1,:] = 0
K[-1,-1] = -1
K[-1,-2] = 1
Q[-1] = -((q/k)*dx**2)

T_diff_finitas_adelantadas = np.linalg.solve(K, Q)
# ------------------------------------

# ------- Pintamos los resultados ------

# Solucion analítica
fig = px.line(
    x=x,
    y=T_analitica,
    labels={"x": "Posición a lo largo de la barra (m)", "y": "Temperatura (°C)"},
    title="Distribución de temperatura a lo largo de la barra con disipación",
)

# Datos de diferencias finitas
fig.add_scatter(
    x=np.linspace(0, L, Nx),
    y=T_diff_finitas_atrasadas,
    mode="markers",
    name="Diferencias Finitas Atrasadas"
)
fig.add_scatter(
    x=np.linspace(0, L, Nx),
    y=T_diff_finitas_centradas,
    mode="markers",
    name="Diferencias Finitas Centradas"
)

fig.add_scatter(
    x=np.linspace(0, L, Nx),
    y=T_diff_finitas_adelantadas,
    mode="markers",
    name="Diferencias Finitas Adelantadas"
)

fig.show()
# ---------------------------------------


## Ejemplo 4: Malla no equiespaciada

In [ ]:
# ------- Solución analítica ------
q = 10 / (L * W * H)  # W/m^3
T_analitica = T_disipacion(x)
# ---------------------------------------------


# Cada elemento de la malla es 2 veces el anterior

Nx = 5

# Número de subdivisiones
Nsubdiv = Nx-1 

dx_min = L/(2**(Nsubdiv)-1)

K = np.zeros((Nx, Nx))
np.fill_diagonal(K, -3)

np.fill_diagonal(K[:-1, 1:], 1)
np.fill_diagonal(K[1:, :-1], 2)


# Condiciones de contorno izquierda
K[0,:] = 0
K[0,0] = 1

K[-1,:] = 0
K[-1,-1] = 1

x_diff  = np.zeros(Nx)
Q = np.zeros(Nx)
for i in range(1, Nx-1):
    dx = dx_min*2**(i-1)
    x_diff[i] = x_diff[i-1] + dx
    Q[i] = (-q/k)*(3*dx**2)

x_diff[-1] = L
Q[0] = T1
Q[-1] = T2


T_diff_finitas = np.linalg.solve(K, Q)

# ------------------------------------

print(list(range(1, 5)))

# ------- Pintamos los resultados ------
# Solucion analítica
fig = px.line(
    x=np.linspace(0, L, 100),
    y=T_analitica,
    labels={"x": "Posición a lo largo de la barra (m)", "y": "Temperatura (°C)"},
    title="Distribución de temperatura a lo largo de la barra con disipación",
)

# Datos de diferencias finitas
fig.add_scatter(
    x=x_diff,
    y=T_diff_finitas,
    mode="markers",
    name="Diferencias Finitas"
)
fig.show()
